This tutorial shows the multi-sectoral capabilities of Macro with synthetic natural gas (SNG) resource. Instead of using a natural gas source node to purchase natural gas for power and hydrogen generators, here a new natural gas (NG) node with demand input from csv file is created. The resources utilized used are PV power, electrolyzer H2, direct air capture (DAC) of CO2, and SNG plant. The system will uses electricity, hydrogen, and atmospheric CO2 to produce SNG to fulfill the NG demand. 

In [1]:
using Pkg
Pkg.activate(".")

  Activating project at `c:\Users\pecci\Code\MACRO-dev\tutorials`


In [2]:
using Macro,CSV,DataFrames

Start MACRO model generation (units: MWh for energy, $ for costs)

In [3]:
T = 8760;
macro_settings = (Commodities = Dict(Electricity=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    Hydrogen=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    NaturalGas=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T),
                                    CO2=>Dict(:HoursPerTimeStep=>1,:HoursPerSubperiod=>T)),
                PeriodLength = T);

In [4]:
macro_settings

(Commodities = Dict{DataType, Dict{Symbol, Int64}}(NaturalGas => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Electricity => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), CO2 => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1), Hydrogen => Dict(:HoursPerSubperiod => 8760, :HoursPerTimeStep => 1)), PeriodLength = 8760)

In [5]:

H2_MWh = 33.33 # MWh per tonne of H2
NG_MWh = 0.29307107 # MWh per MMBTU of NG

df = CSV.read("time_series_data_for_synthetic_natural_gas_tutorial.csv",DataFrame)

electricity_demand = df[1:T,:Electricity_Demand_MW]; # MWh
solar_capacity_factor = df[1:T,:Solar_Capacity_Factor]; # factor between 0 and 1
h2_demand = H2_MWh*df[1:T,:H2_Demand_tonne]; # MWh of hydrogen
ng_demand = NG_MWh*df[1:T,:NG_Demand_MMBtu]; # MWh of natural gas

solar_inv_cost = 85300; # $/MW
solar_fom_cost = 18760.0; # $/MW

battery_inv_cost = 19584.0; #$/MW
battery_fom_cost = 4895; # $/MW
battery_vom_cost = 0.15; #$/MW
battery_inv_cost_storage = 22494.0; #$/MWh
battery_fom_cost_storage = 5622; #$/MWh
battery_vom_cost_storage = 0.15; #$/MWh
battery_eff_up = 0.92;
battery_eff_down = 0.92;

electric_dac_inv_cost = 9390;# $/Tonnes of CO2 captured/h
electric_dac_fom_cost = 7470;# $/Tonnes of CO2 captured/h
electric_dac_vom_cost = 22; #$/Tonnes of CO2 captured
electric_dac_power_input = 4.38; #MWh/Tonnes of CO2 captured
electric_dac_capsize = 20;# Tonnes of CO2 captured/h

sng_inv_cost = 390;# $/MW of NG
sng_fom_cost = 470;# $/MW of NG
sng_vom_cost = 2; #$/MWh of NG
sng_power_input = 2; #MWh of power/MWh of NG
sng_h2_input = 2; #MWh of H2/MWh of NG
sng_co2_input = 2; #Tonnes of CO2 captured/MWh of NG
sng_capsize = 20;# MW of NG

electrolyzer_capsize = 2*H2_MWh; # MWh of H2
electrolyzer_efficiency = 1/(45/H2_MWh); # MWh of H2 / MWh of electricity
electrolyzer_inv_cost = 2033333/H2_MWh; # $/MW of H2
electrolyzer_fom_cost = 30500/H2_MWh; # $/MW of H2
electrolyzer_vom_cost = 0.0;


In [6]:
### For now, we use the same temporal resolution and subperiods for every commodity, but MACRO will allow to use different temporal resolution for different commodities

hours_per_timestep(c) = macro_settings.Commodities[c][:HoursPerTimeStep];
hours_per_subperiod(c) = macro_settings.Commodities[c][:HoursPerSubperiod]
time_interval(c)= 1:hours_per_timestep(c):macro_settings.PeriodLength

subperiods(c) = collect(Iterators.partition(time_interval(c), Int(hours_per_subperiod(c) / hours_per_timestep(c))),)

subperiods (generic function with 1 method)

Let's start from creating an electricity node:

In [7]:
e_node = Node{Electricity}(;
    id = Symbol("E_node"),
    demand =  electricity_demand,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    max_nsd = [0.0],
    price_nsd = [50000],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We assign a solar PV generator to this electricity node:

In [8]:
solar_pv = Resource{Electricity}(;
    id = :Solar_PV,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    capacity_factor = solar_capacity_factor,
    can_expand = true, 
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = solar_inv_cost,
    fixed_om_cost = solar_fom_cost,
    constraints = [Macro.CapacityConstraint()]
)

Resource{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :Solar_PV, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1779, 0.429  …  0.3834, 0.2594, 0.078, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1.0, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, 0.0, true, false, 85300.0, 18760.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])

We can also assigne a battery storage to the node:

In [9]:
battery = SymmetricStorage{Electricity}(;
    id = :battery,
    node = e_node,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    can_expand = true,
    can_retire = false,
    existing_capacity = 0.0,
    investment_cost = battery_inv_cost,
    investment_cost_storage = battery_inv_cost_storage,
    fixed_om_cost = battery_fom_cost,
    fixed_om_cost_storage = battery_fom_cost_storage,
    variable_om_cost = battery_vom_cost,
    variable_om_cost_storage = battery_vom_cost_storage,
    efficiency_injection = battery_eff_down,
    efficiency_withdrawal = battery_eff_up,
    constraints = [Macro.CapacityConstraint(),Macro.StorageCapacityConstraint()]
)

SymmetricStorage{Electricity}(Node{Electricity}(:E_node, [7850.0, 7424.0, 7107.0, 6947.0, 6922.0, 7045.0, 7307.0, 7544.0, 7946.0, 8340.0  …  10438.0, 10469.0, 11228.0, 11908.0, 11562.0, 9923.0, 9461.0, 9018.0, 8551.0, 8089.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [50000.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :battery, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0.0, Inf, 0.0, Inf, 0.0, 0.0, true, false, 19584.0, 22494.0, 0.0, 4895.0, 5622.0, 0.0, 0.15, 0.15, 0.0, 0.92, 0.92, 0.0, 1.0, 10.0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing), Macro.StorageCapacityConstraint(missing, missing, missing)])

We also consider a hydrogen demand node:

In [10]:
h2_node = Node{Hydrogen}(;
    id = Symbol("H2_node"),
    demand =  h2_demand,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We also consider a natural gas demand node:

In [11]:
ng_node = Node{NaturalGas}(;
    id = Symbol("NG_node"),
    demand =  ng_demand,
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    max_nsd = [0.0],
    price_nsd = [0.0],
    constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

Node{NaturalGas}(:NG_node, [0.4747751334, 0.3575467054, 0.5128743725, 1.17228428, 2.4149056168, 3.9300830487, 4.9206632653, 5.638687386799999, 5.931758456799999, 6.2805130301  …  7.930503154199999, 7.6022635558000005, 7.212479032699999, 6.3772264832, 5.3104477884, 4.0473114767, 3.077246235, 2.1452802324, 1.3276119471, 0.7854304676], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)])

We also define a CO2 node with sink and source:

In [12]:
co2_node = Node{CO2}(;
id = Symbol("CO2_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_sink = Sink{CO2}(;
node = co2_node,
id = Symbol("CO2_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)

co2_source = Resource{CO2}(;
id = :CO2_Source,
node = co2_node,
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
can_expand = false, 
can_retire = false,
price = zeros(length(time_interval(CO2))),
existing_capacity = Inf,
)

Resource{CO2}(Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_Source, Float64[], 1.0, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], 0.0, Inf, Inf, false, false, 0.0, 0.0, 0.0, Dict{Symbol, Any}(), Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We define captured CO2 node and sink to account for the inventory of captured CO2 (input from CO2 capture, output to SNG or potential storage):

In [13]:
co2_captured_node = Node{CO2}(;
id = Symbol("CO2_Captured_node"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
demand = zeros(length(time_interval(CO2))),
max_nsd = [0.0],
price_nsd = [0.0],
constraints = [Macro.DemandBalanceConstraint(),Macro.MaxNonServedDemandConstraint()]
)

co2_captured_sink = Sink{CO2}(;
node = co2_captured_node,
id = Symbol("CO2_sink"),
time_interval = time_interval(CO2),
subperiods = subperiods(CO2),
)



Sink{CO2}(Node{CO2}(:CO2_Captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), :CO2_sink, Float64[], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], Dict{Symbol, Any}(), Macro.AbstractTypeConstraint[])

We can also create an electrolyzer, which is defined by a single stoichiometry balance: electrolyzer_efficiency $\times$ Electricity = H2

In [14]:
electrolyzer = Transformation{Electrolyzer}(;
                id = :Electrolyzer,
                time_interval = time_interval(Electricity),
                stoichiometry_balance_names = [:energy],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electrolyzer.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = electrolyzer,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electrolyzer_capsize,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:energy=>1.0),
    existing_capacity = 0.0,
    investment_cost = electrolyzer_inv_cost,
    fixed_om_cost = electrolyzer_fom_cost,
    variable_om_cost = electrolyzer_vom_cost,
    constraints = [CapacityConstraint()]
)

electrolyzer.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electrolyzer,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>electrolyzer_efficiency)
)

2-element Vector{TEdge}:
 TEdge{Hydrogen}(:H2, Node{Hydrogen}(:H2_node, [53.9946, 40.6626, 58.3275, 133.32, 274.6392, 446.95529999999997, 559.6107, 641.2692, 674.5991999999999, 714.2619  …  901.9097999999999, 864.5802, 820.2512999999999, 725.2608, 603.9396, 460.2873, 349.965, 243.9756, 150.9849, 89.3244], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{Electrolyzer}(:Electrolyzer, 1:1:8760, 1, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 66.66, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0], 0.0, Inf, 0.0, 61006.090609060906, 915.0915091509152, 0.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConst

Let's add a direct air capture unit consuming electricity and capturing CO2 from the atmosphere into the captured CO2 inventory

In [15]:
electric_dac = Transformation{DACElectric}(;
                id = :Electric_DAC,
                time_interval = time_interval(CO2),
                stoichiometry_balance_names  = [:energy,:captured_co2],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

electric_dac.TEdges[:CO2_Captured] = TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = electric_dac,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = electric_dac_capsize,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>electric_dac_power_input,:captured_co2=>1.0),
    existing_capacity = 0.0,
    investment_cost = electric_dac_inv_cost,
    fixed_om_cost = electric_dac_fom_cost,
    variable_om_cost = electric_dac_vom_cost,
    constraints = [CapacityConstraint()]
)

electric_dac.TEdges[:CO2_input] = TEdge{CO2}(;
    id = :CO2_input,
    node = co2_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:energy=>0.0,:captured_co2=>1.0)
)

electric_dac.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = electric_dac,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:energy=>1.0,:captured_co2=>0.0)
)

3-element Vector{TEdge}:
 TEdge{CO2}(:CO2_Captured, Node{CO2}(:CO2_Captured_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{DACElectric}(:Electric_DAC, 1:1:8760, 2, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 20.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [1.0, 4.38], 0.0, Inf, 0.0, 9390.0, 7470.0, 22.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[CapacityConstraint(missing, missing, missing)])
 TEdge{CO2}(:CO2_Atmosphere, Node{CO2}(:CO2_node, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

Let's add a synthetic natural gas unit consuming electricity, hydrogen, and captured CO2 to produce natural gas output

In [16]:
sng = Transformation{SyntheticNG}(;
                id = :SNG,
                time_interval = time_interval(NaturalGas),
                stoichiometry_balance_names  = [:electricity,:hydrogen,:co2],
                constraints = [Macro.StoichiometryBalanceConstraint()]
                )

sng.TEdges[:NaturalGas] = TEdge{NaturalGas}(;
    id = :NaturalGas,
    node = ng_node,
    transformation = sng,
    direction = :output,
    has_planning_variables = true,
    can_expand = true,
    can_retire = false,
    capacity_size = sng_capsize,
    time_interval = time_interval(NaturalGas),
    subperiods = subperiods(NaturalGas),
    st_coeff = Dict(:power=>sng_power_input,:hydrogen=>sng_h2_input,:co2=>sng_co2_input),
    existing_capacity = 0.0,
    investment_cost = sng_inv_cost,
    fixed_om_cost = sng_fom_cost,
    variable_om_cost = sng_vom_cost,
    constraints = [CapacityConstraint()]
)

sng.TEdges[:E] = TEdge{Electricity}(;
    id = :E,
    node = e_node,
    transformation = sng,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Electricity),
    subperiods = subperiods(Electricity),
    st_coeff = Dict(:power=>1.0,:hydrogen=>0.0,:co2=>0.0)
)

        
sng.TEdges[:H2] = TEdge{Hydrogen}(;
    id = :H2,
    node = h2_node,
    transformation = sng,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(Hydrogen),
    subperiods = subperiods(Hydrogen),
    st_coeff = Dict(:power=>0.0,:hydrogen=>1.0,:co2=>0.0)
)

sng.TEdges[:CO2_Captured] = TEdge{CO2}(;
    id = :CO2_Captured,
    node = co2_captured_node,
    transformation = sng,
    direction = :input,
    has_planning_variables = false,
    time_interval = time_interval(CO2),
    subperiods = subperiods(CO2),
    st_coeff = Dict(:power=>0.0,:hydrogen=>0.0,:co2=>1.0)
)



4-element Vector{TEdge}:
 TEdge{NaturalGas}(:NaturalGas, Node{NaturalGas}(:NG_node, [0.4747751334, 0.3575467054, 0.5128743725, 1.17228428, 2.4149056168, 3.9300830487, 4.9206632653, 5.638687386799999, 5.931758456799999, 6.2805130301  …  7.930503154199999, 7.6022635558000005, 7.212479032699999, 6.3772264832, 5.3104477884, 4.0473114767, 3.077246235, 2.1452802324, 1.3276119471, 0.7854304676], 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [0.0], [0.0], Dict{Any, Any}(), Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.DemandBalanceConstraint(missing, missing, missing), Macro.MaxNonServedDemandConstraint(missing, missing, missing)]), Transformation{SyntheticNG}(:SNG, 1:1:8760, 3, TEdge[#= circular reference @-3 =#], Dict{Any, Any}(), Macro.AbstractTypeConstraint[Macro.StoichiometryBalanceConstraint(missing, missing, missing)]), :output, true, false, true, 20.0, 1:1:8760, StepRange{Int64, Int64}[1:1:8760], [2.0, 2.0, 2.0], 0.0, Inf, 0.0, 390.0, 470.0, 2.0, 0.0, false, 0.0, 0.0, 0, 0, 0.0, 

Now that we have implemented different technologies, we can create the JuMP model.

In [17]:
components = [solar_pv; battery; electrolyzer; electric_dac; sng; co2_sink;co2_source;co2_captured_sink];
nodes = [e_node; h2_node; ng_node; co2_node; co2_captured_node];
system = [nodes;components];

In [18]:
model = Macro.JuMP.Model();
Macro.@variable(model,vREF==1) ## Variable used to initialize empty expressions
Macro.@expression(model, eFixedCost, 0 * model[:vREF]);
Macro.@expression(model, eVariableCost, 0 * model[:vREF]);

In [19]:
add_planning_variables!.(components,model);

In [20]:
add_operation_variables!.(system, model);

In [21]:
add_all_model_constraints!.(system, model);

In [22]:
Macro.@objective(model, Min, model[:eFixedCost] + model[:eVariableCost]);


In [23]:
#Macro.@constraint(model,CO2Cap,sum(Macro.outflow(co2_captured_node)[t] for t in time_interval(CO2_Captured)) == 100000);

In [24]:
println(electrolyzer.constraints[1].constraint_ref[:energy,5])

vFLOW_Hydrogen_H2[5] - 0.7406666666666667 vFLOW_Electricity_E[5] == 0


In [25]:
println(electric_dac.constraints[1].constraint_ref[:captured_co2,5])

vFLOW_CO2_CO2_Captured[5] - vFLOW_CO2_CO2_Atmosphere[5] == 0


In [26]:
println(electric_dac.constraints[1].constraint_ref[2,5])

4.38 vFLOW_CO2_CO2_Captured[5] - vFLOW_Electricity_E[5] == 0


In [27]:
println(sng.constraints[1].constraint_ref[1,5])

2 vFLOW_NaturalGas_NaturalGas[5] - vFLOW_Electricity_E[5] == 0


In [28]:
println(sng.constraints[1].constraint_ref[2,5])

2 vFLOW_NaturalGas_NaturalGas[5] - vFLOW_Hydrogen_H2[5] == 0


In [29]:
println(sng.constraints[1].constraint_ref[3,5])

2 vFLOW_NaturalGas_NaturalGas[5] - vFLOW_CO2_CO2_Captured[5] == 0


In [30]:
using Gurobi

In [31]:
Macro.set_optimizer(model,Gurobi.Optimizer)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-03-14


In [32]:
Macro.optimize!(model)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i9-9980HK CPU @ 2.40GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 201488 rows, 183979 columns and 459771 nonzeros
Model fingerprint: 0x8845cf90
Coefficient statistics:
  Matrix range     [1e-04, 7e+01]
  Objective range  [1e-01, 4e+06]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e-01, 2e+04]
Presolve removed 162127 rows and 153374 columns
Presolve time: 0.14s
Presolved: 39361 rows, 30605 columns, 132865 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 Dense cols : 4
 AA' NZ     : 1.250e+05
 Factor NZ  : 5.333e+05 (roughly 30 MB of memory)
 Factor Ops : 8.095e+06 (less than 1 second per iteration)
 Threads    : 6

                  Objective                Residual
Iter       Primal          Dual      

The installed electrolyzer capacity is:

In [33]:
Macro.value(Macro.capacity(electrolyzer.TEdges[1]))

1057.8444733466

The installed solar capacity in MW is:

In [34]:
Macro.value(Macro.capacity(solar_pv))

133848.42527364765

The installed battery capacity in MW is:

In [35]:
Macro.value(Macro.capacity(battery))

16431.746098987787

The installed Electric DAC capacity in Tons CO2/h is:

In [36]:
Macro.value(Macro.capacity(electric_dac.TEdges[1]))

18.2817733466

The installed SNG capacity in MW NG is:

In [37]:
Macro.value(Macro.capacity(sng.TEdges[1]))

9.1408866733

The resulting CO2 removed from the atmosphere by DAC in tonnes are:

In [38]:
atm_CO2_captured  = Macro.value(sum(Macro.injection(co2_source)[t] for t in time_interval(CO2)))

79189.19227087137